In [68]:
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import (
    MinMaxScaler,
    PolynomialFeatures,
)
from sklearn.feature_selection import (
    VarianceThreshold
)
from sklearn.decomposition import PCA

from sklearn.dummy import DummyClassifier

from sklearn.datasets import fetch_openml

import pandas as pd
import numpy as np

In [69]:
Xraw,y = fetch_openml(data_id=1461,return_X_y=True,as_frame=True)

c:\Users\Jason\Desktop\git_control\ml-research\.venv\Lib\site-packages\sklearn\datasets\_openml.py:968: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [70]:
Xraw.columns = [
    'age','job','marital','education','default','balance','housing','loan','contact','day','month','duration','campaign','pdays','previous','poutcome'
]

In [71]:
y.name='subscribed'

In [72]:
X=pd.DataFrame()
X[['age','balance','duration','campaign','pdays','previous']] = Xraw[
    ['age','balance','duration','campaign','pdays','previous']
].copy()

In [73]:
for c in [
    'job','marital','education','default','housing','loan','contact','month','poutcome'
]:
    bins = pd.get_dummies(Xraw[c],drop_first=True,prefix=c,dtype=int)

    X[bins.columns] = bins.copy()
    


In [74]:
Xtrain,Xtest,ytrain,ytest = train_test_split(
    X,y,test_size=.1,shuffle=True,random_state=42
)


In [129]:
model = Pipeline(
    steps=[
        ('poly',PolynomialFeatures(degree=2,interaction_only=False,include_bias=True)),
        ('variance',VarianceThreshold(threshold=0)),
        ('pca',PCA(n_components=20)),
        ('scaler',MinMaxScaler()),
        ('learner',DummyClassifier(random_state=42,strategy='stratified'))
        #('learner',RandomForestClassifier(random_state=42))
    ]
)

model.fit(Xtrain,ytrain)
fit = pd.Series(model.predict(Xtrain),index = Xtrain.index)
pred = pd.Series(model.predict(Xtest),index = Xtest.index)

result = pd.DataFrame({
    'y':ytest,
    'pred':pred
})
result['cnt'] = 1

pd.crosstab(columns = result['pred'],index=result['y'],values = result['cnt'],aggfunc='sum')

pred,1,2
y,,
1,3532,436
2,486,68


In [130]:
model = Pipeline(
    steps=[
        ('poly',PolynomialFeatures(degree=2,interaction_only=False,include_bias=True)),
        ('variance',VarianceThreshold(threshold=0)),
        ('pca',PCA(n_components=40)),
        ('scaler',MinMaxScaler()),
        #('learner',DummyClassifier(random_state=42,strategy='uniform'))
        ('learner',RandomForestClassifier(random_state=42))
    ]
)

model.fit(Xtrain,ytrain)
fit = pd.Series(model.predict(Xtrain),index = Xtrain.index)
pred = pd.Series(model.predict(Xtest),index = Xtest.index)

result = pd.DataFrame({
    'y':ytest,
    'pred':pred
})
result['cnt'] = 1

conf_mat = pd.crosstab(columns = result['pred'],index=result['y'],values = result['cnt'],aggfunc='sum')



In [131]:
conf_mat

pred,1,2
y,,
1,3862,106
2,380,174


In [132]:
conf_mat.divide(conf_mat.sum(axis=1),axis=0)

pred,1,2
y,,
1,0.973286,0.026714
2,0.685921,0.314079


In [133]:
conf_mat.divide(conf_mat.sum(axis=0),axis=1)

pred,1,2
y,,
1,0.91042,0.378571
2,0.08958,0.621429


In [141]:
(result['y']==result['pred']).mean()

0.8925254312251216